In [1]:
import pandas as pd
import tensorflow as tf

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

In [2]:
import random
import numpy as np
import math
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

from deap import benchmarks

In [3]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [4]:
toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=65)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [5]:
DATASET_TEST_PATH = './/AIRTLab//test//'

DATASET_TRAIN_PATH = './/AIRTLab//train//'

TARGET_SIZE = (120,160)

DESIRED_ACCURACY = 0.995

UPPER_BOUND = 155

LOWER_BOUND = 90

In [6]:
def test_dataset():
    train_datagen = ImageDataGenerator(rescale = 1.0/255.)

    return train_datagen.flow_from_directory(DATASET_TEST_PATH,
                                             batch_size=32,
                                             class_mode='categorical',
                                             target_size=TARGET_SIZE)

def train_dataset():
    train_datagen = ImageDataGenerator(rescale = 1.0/255.)

    return train_datagen.flow_from_directory(DATASET_TRAIN_PATH,
                                             batch_size = 32,
                                             class_mode = 'categorical',
                                             target_size =TARGET_SIZE)

In [7]:
train_set = train_dataset()
val_set = test_dataset()

Found 2926 images belonging to 2 classes.
Found 1252 images belonging to 2 classes.


In [8]:
lista_resultados_treinos = []

In [9]:
class accCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

In [10]:
import gc

class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [11]:
def evaluate_individual(trainable_list):

  pre_trained_model = MobileNetV2(input_shape = (120, 160, 3), include_top = False)

  last_layer = pre_trained_model.get_layer('out_relu')
  last_output = last_layer.output

  #layers_to_set = pre_trained_model.layers

  for layer in pre_trained_model.layers:
        layer.trainable = False

  for layer, trainable in zip(pre_trained_model.layers[LOWER_BOUND:UPPER_BOUND], trainable_list):
    layer.trainable = bool(trainable)
    #print(layer.trainable)

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)
  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(1024, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x)
  # Add a final sigmoid layer for classification
  x = layers.Dense(2, activation='softmax')(x)

  model = Model(pre_trained_model.input, x)
  model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics='accuracy')

  history = model.fit(
    train_set,
    epochs=1,
    batch_size=32,
    validation_data=val_set,
    callbacks=[accCallback(),GarbageCollectorCallback()]
  )

  lista_resultados_treinos.extend([pd.Series({key: history.history[key][i] for key in history.history}) for i in range(len(history.history['loss']))])
  result = (history.history['val_accuracy'][0],)
  tf.keras.backend.clear_session()
  
  return  result #return validation accuracy last result

In [12]:
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxUniformPartialyMatched, indpb=0.1) 
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament,tournsize=2) 

In [13]:
def main():
  random.seed(64)

  pop = toolbox.population(n=8)
  hof = tools.HallOfFame(1)
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", np.mean)
  stats.register("std", np.std)
  stats.register("min", np.min)
  stats.register("max", np.max)


  pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=4,
                                   stats=stats, halloffame=hof, verbose=True)

  return pop, log, hof

In [14]:
if __name__ == "__main__":
  results = main()

  gen = results[1].select('gen')
  best_per_gen = results[1].select('max')

  #print(results[2])





92/92 [==============================] - 49s 505ms/step - loss: 0.4557 - accuracy: 0.9405 - val_loss: 0.3535 - val_accuracy: 0.8602
gen	nevals	avg     	std     	min    	max     
0  	8     	0.807408	0.102053	0.59345	0.940895
92/92 [==============================] - 51s 530ms/step - loss: 0.3097 - accuracy: 0.9375 - val_loss: 0.9564 - val_accuracy: 0.7971
1  	4     	0.840256	0.109771	0.646965	0.940895
92/92 [==============================] - 52s 540ms/step - loss: 0.4122 - accuracy: 0.9299 - val_loss: 0.4902 - val_accuracy: 0.8746
2  	7     	0.81899 	0.0569645	0.749201	0.912141
92/92 [==============================] - 53s 548ms/step - loss: 0.3973 - accuracy: 0.9416 - val_loss: 0.3165 - val_accuracy: 0.9233
3  	6     	0.747504	0.122589 	0.505591	0.923323
92/92 [==============================] - 51s 528ms/step - loss: 0.4159 - accuracy: 0.9289 - val_loss: 0.4518 - val_accuracy: 0.8530
4  	3     	0.795128	0.0874409	0.617412	0.923323


In [17]:
generations = results[1]

In [18]:
df = pd.DataFrame(generations)
df.to_csv('results_comp_evo.csv', index=False)

In [19]:
pd.DataFrame(lista_resultados_treinos).to_csv('lista_resultados.csv', index=False)

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('results_comp_evo.csv', index_col='gen')
df.head()

,nevals,avg,std,min,max
gen,,,,,
0,8,0.807408,0.102053,0.593450,0.940895
1,4,0.840256,0.109771,0.646965,0.940895
2,7,0.818990,0.056965,0.749201,0.912141
3,6,0.747504,0.122589,0.505591,0.923323
4,3,0.795128,0.087441,0.617412,0.923323
